# Syntenic Coordinate Shift

## Project information

This notebook is essentially a set of functions that will allow for the conversion of one genomes coordinates to another via their syntenic relationship (if it exists)

The program utilizes the syntenic alignment outputs from the program Satsuma to shift coordinates from one genome to another.  
Syntenic blocks between the two input genomes are found and defined by Satsuma and written out to two main files...  
**satsuma_summary.chained.out** - The most succinct version of the sytenic blocks with only the coordinates and orientation of the two regions with respect to eachother.  
**MergeXCorrMatches.chained.out** - The same information as the summary.chained.out file, but contains the actual alignment that was found between the two regions. **This is the file that is used as input into the coordinate shifting algorithm.**

The program works in two steps (with a bit more detailed descriptions later in the notebook) -  
1. Read the reference genome scaffold/chromosome sequences into memory (fastaFilePaths)  
   Read the MergeXCorrMatches.chained.out file into memory in the form of two interval trees (one for each target and queary genomes) with adjusted genomic coordinates based on the reference genomes  
2. Queary any given Chromosome,position in one genome to find the sytenic region and corresponding base pair in the other genome. There are two functions that accomplish this...  
-The first is **quearyTree** wich will identify the sytenic region/block(s) that the position falls within.  
-The second is **quearySynBlock** which will take the SynBlock object found in the first function, and find the coordinate/base pair information found between genome1 and genome2.

Essentially, the first step of the algorithm takes about 5-10 minutes to run depending on the machine but only needs to be run once eachtime the notebook/script is opened. Then all point quearies are super fast.  
The alternative to this would be a database, but there are numerous reasons why we chose not to go that route

## Imports

In [1]:
import time
import random
import os
import intervaltree
import pickle

## Satsuma homology to memory

In [2]:
def readFastaToMem(fastaFile):
    inFile = open(fastaFile,'r')
    chromDict = {}
    for line in inFile:
        line = line.strip('\r').strip('\n')
        if line[0] == ">":
            chrom = line[1:]
            chromDict.update({ chrom:[] })
        else:
            chromDict[chrom].append(line)
    ##############
    inFile.close()
    for c in chromDict:
        chromDict[c] = ''.join(chromDict[c])
    ################
    return chromDict

def parseIntervalLines(inFile,line1,line2):
    cols1 = line1.strip('\r').strip('\n').split(' ')
    cols2 = line2.strip('\r').strip('\n').split(' ')
    
    orientation = cols1[7]
    length = int(cols1[9])
    quearyChrom,targetChrom = cols1[1],cols1[5]
    quearyInterval = cols1[2].strip(']').strip('[').split('-')
    quearyStart,quearyStop = int(quearyInterval[0]),int(quearyInterval[1])
    targetInterval = cols1[6].strip(']').strip('[').split('-')
    targetStart,targetStop = int(targetInterval[0]),int(targetInterval[1])
    pID = float(cols2[4])
    #if abs(targetStart-targetStop) != abs(quearyStart-quearyStop):
    #    print(line)

    
    synBlockObj = SynBlock(quearyChrom,quearyStart,quearyStop,
                           targetChrom,targetStart,targetStop,
                           orientation,length,pID)
    
    inFile.readline()
    inFile.readline()
    
    return inFile,synBlockObj

class SynBlock:
    def __init__(self,quearyName,quearyBegin,quearyEnd,targetName,targetBegin,targetEnd,orientation,length,pID):
        self.quearyName = quearyName
        self.quearyBegin = quearyBegin
        self.quearyEnd = quearyEnd
        self.targetName = targetName
        self.targetBegin = targetBegin
        self.targetEnd = targetEnd
        self.orientation = orientation
        self.length = length
        self.pID = pID
    ############################
    def getSequences(self,seqs):
        self.quearySeq = ''.join(seqs[0])
        self.linkSeq = ''.join(seqs[1])
        self.targetSeq = ''.join(seqs[2])

def processAlignment(inFile,alignment):
    alignment[1].append(inFile.readline().strip('\r').strip('\n'))
    alignment[2].append(inFile.readline().strip('\r').strip('\n'))
    
    inFile.readline()
    inFile.readline()
    inFile.readline()
    inFile.readline()
    
    return inFile,alignment

def revCompSatsuma(seq):
    compDict = { "A":"T","T":"A","G":"C","C":"G",
                 "a":"t","t":"a","g":"c","c":"g",
                 "N":"N","n":"n","-":"-" }
    ##################################################
    return ''.join([ compDict[s] for s in seq ][::-1])

def preProcessIntervalCoords(synBlockObj,quearyRefDict,targetRefDict,howFar=2000):
    qSeq = ''.join([ s for s in synBlockObj.quearySeq if s != "-" ])
    if synBlockObj.orientation == "-":
        qSeq = revCompSatsuma(qSeq)
    qSeqLength,success,changed = len(qSeq),0,0
    ########################
    for i in range(0,howFar):
        beginF = synBlockObj.quearyBegin+i
        beginR = synBlockObj.quearyBegin-i
        refSeqF = quearyRefDict[synBlockObj.quearyName][beginF:beginF+qSeqLength]
        refSeqR = quearyRefDict[synBlockObj.quearyName][beginR:beginR+qSeqLength]
        if qSeq == refSeqF:
            success = 1
            synBlockObj.quearyBegin = beginF
            synBlockObj.quearyEnd = beginF+qSeqLength
            if i != 0:
                changed += 1
            break
        #####################
        elif qSeq == refSeqR:
            success = 1
            synBlockObj.quearyBegin = beginR
            synBlockObj.quearyEnd = beginR+qSeqLength
            if i != 0:
                changed += 1
            break
    ################
    if success == 0:
        print("### ERROR ###")
        print(synBlockObj.quearyName,synBlockObj.quearyBegin,synBlockObj.quearyEnd)
        print(synBlockObj.length,synBlockObj.pID,synBlockObj.orientation)
        return synBlockObj,changed
    else:
        return synBlockObj,changed
    
def readXHomology_toSynBlocks(XFile):
    '''Takes in the MergeXCorrMatches.chained.out produced by Satsuma as input.
       A SynBlock object is created for each sytenic pair found by satsuma and placed in
       interval trees pertaining to the queary/target genomes. It's worth noting that the 
       target and queary interval trees both point to the same SynBlock object.
       ############################## 
       XFile = filePath/To/MergeXFile
       Output = [ synBlockObject , ... ] '''
    inFile = open(XFile,'r')
    ### Read header lines ###
    for i in range(0,12):
        inFile.readline()
    ##############
    synBlocks = []
    synBlock = "Dummy variable for the initialization of the alg... Is overwritten later. Not to worry"
    alignment = [ [],[],[] ]
    aLCount = 1
    while True:
        lineMan = inFile.readline().strip('\r').strip('\n')
        lineInfo = lineMan.split(' ')[0]
        #######################
        if lineInfo == "Found":
            if len(alignment[0]) != 0:
                synBlock.getSequences(alignment)
                synBlocks.append(synBlock)
            aLCount += 1
            alignment = [ [],[],[] ]
            infoLine = inFile.readline()
            infoLine2 = inFile.readline()
            inFile,synBlock = parseIntervalLines(inFile,infoLine,infoLine2 )   
        #########################
        elif lineInfo == "Query":
            if len(alignment[0]) != 0:
                synBlock.getSequences(alignment)
                synBlocks.append(synBlock)
            aLCount += 1
            alignment = [ [],[],[] ]
            infoLine2 = inFile.readline()
            inFile,synBlock = parseIntervalLines(inFile,lineMan,infoLine2 )   
        ###################
        elif lineMan == '':
            e1 = inFile.readline().strip('\r').strip('\n') ### Link line
            e2 = inFile.readline().strip('\r').strip('\n') ### Sequence2 line
            e3 = inFile.readline().strip('\r').strip('\n') ### Spacer line
            e4 = inFile.readline().strip('\r').strip('\n') ### "Protein coding line"          
            if "Total" not in e1:
                alignment[1].append(e1)
                alignment[2].append(e2)
                inFile.readline()
                inFile.readline()
                
            else:
                print("End of file")
                print("Printing satsuma stats (last few lines of the MergeXHomology file)...")
                synBlock.getSequences(alignment)
                synBlocks.append(synBlock)
                aLCount += 1
                alignment = [ [],[],[] ]
                break
        #####
        else:
            alignment[0].append(lineMan)
            inFile,alignment = processAlignment(inFile,alignment)
        ##########################
        if aLCount % 1000000 == 0:
            print("Alignments processed "+str(aLCount))
    ##############
    inFile.close()
    #########
    print(e1)
    print(e2)
    print(e3)
    print(e4)
    print("Alignments put into interval tress "+str(aLCount))
    ################
    return synBlocks

def synBlocks_toIntervalTrees(synBlocks,qRef,tRef,howFar=2000):
    ### howFar = Maximum distance to search in the quearyReference genome left/right of the quearyBegin position 
    altered = 0
    qTrees,tTrees = {},{}
    for i,sObj in enumerate(synBlocks):
        if sObj.quearyName not in qTrees:
            qTrees.update({ sObj.quearyName:intervaltree.IntervalTree() })
        
        if sObj.targetName not in tTrees:
            tTrees.update({ sObj.targetName:intervaltree.IntervalTree() })
        ##################################################################
        
        synBlock,c = preProcessIntervalCoords(sObj,qRef,tRef,howFar=howFar)
        altered += c
        
        qTrees[sObj.quearyName].addi(sObj.quearyBegin,sObj.quearyEnd,sObj)
        tTrees[sObj.targetName].addi(sObj.targetBegin,sObj.targetEnd,sObj)
        
        if i % 500000 == 0:
            print("Synblocks procssed "+str(i)+"...")
    ###########################################################
    print("Syntenic blocks with adjusted coords "+str(altered))
    return qTrees,tTrees,synBlocks       

**The general flow of the program is as follows**


1. Read the queary and target reference genomes into memory in the form of { scaffoldName:scaffoldSequence } (one hashTable for each genome)  
2. Read the satsuma MergeXHomology file into memory by storing the information in the form of a SynBlock Object. This is just an easy way to store/access all the information  
3. "Pre-process" the SyntenyBlock Objects, by changing the quearyBegin and quearyEnd attributes if the satsuma coordinates are wrong.  
We do this by matching up the quearySequence (output by satsuma) to the respective reference genome sequence, and pull the coordinates (if they have changed)  
This is only an issue for the Queary genome that was used in the Satsuma run that generated all the data.  
4. Add the queary and target information to their respective intervalTrees for a fast lookup  
5. Any given queary/target genome position is now quearyable to obtain its sytenic partner

In [3]:
startTime = time.time()
marmDict = readFastaToMem("/fsimb/groups/imb-baumanngr/ao/HYBRID_COORDINATES/a_marmorata_renamed.fasta")
gulDict = readFastaToMem("/fsimb/groups/imb-baumanngr/ao/HYBRID_COORDINATES/a_gularis_unmasked_fixed.fasta")
print("Time to read fasta sequences to memory "+str(time.time()-startTime)+" seconds")

Time to read fasta sequences to memory 16.03819990158081 seconds


In [4]:
startTime = time.time()
synBlocks = readXHomology_toSynBlocks("/fsimb/groups/imb-baumanngr/ao/HYBRID_COORDINATES/MergeXCorrMatches.chained.out")
print("Time to read satsuma alignments to syntenic blocks "+str(time.time()-startTime)+" seconds")

Alignments processed 1000000
Alignments processed 1000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
Alignments processed 2000000
End of file
Printing satsuma stats (last few lines of the MergeXHomology file)...
Total bases aligned:               1230755977
Total bases in target sequence:    1498801201
Covered by at least one alignment: 1199035106 (79.9996 %)
All bases in aligns: 2016206081
Alignments put into interval tress 2311565
Time to read satsuma alignments to syntenic blocks 48.01776909828186 seconds


In [5]:
startTime = time.time()
Q,T,synBlocks = synBlocks_toIntervalTrees(synBlocks,qRef=marmDict,tRef=gulDict,howFar=2000)
print("Time to adjust coordinates and build interval tress "+str(time.time()-startTime)+" seconds")

Synblocks procssed 0...
Synblocks procssed 500000...
Synblocks procssed 1000000...
Synblocks procssed 1500000...
Synblocks procssed 2000000...
Syntenic blocks with adjusted coords 59370
Time to adjust coordinates and build interval tress 289.77008605003357 seconds


## Coordinate shift code

In [6]:
### From the satsuma manual/documentation 
### http://satsuma.sourceforge.net/faq.html

### in principle, the TARGET should be the more COMPLETE genome

### Satsuma chained out format = 
### Target sequence name (provided by fasta)
### First target base
### Last target base
### Query sequence name (provided by fasta)
### First query base
### Last query base
### Identity orientation

In [7]:
def printInfo(sObj):
    print(sObj.quearyName,sObj.quearyBegin,sObj.quearyEnd)
    print(sObj.targetName,sObj.targetBegin,sObj.targetEnd)
    print("Orientation ( "+str(sObj.orientation)+" )")
    print("PercentIdentity:"+str(sObj.pID))
    print("Length:"+str(sObj.length))
    print("#####")

def printAln(sObj,step=100):
    for i in range(0,sObj.length,step):
        print(sObj.quearySeq[i:i+step])
        print(sObj.linkSeq[i:i+step])
        print(sObj.targetSeq[i:i+step])
        print()
        print(''.join(["-"]*step))
        print()

def quearySynBlock(synBlockObj,chrom,point,which="queary"):
    if synBlockObj.orientation == "+":
        qCoord,tCoord = synBlockObj.quearyBegin,synBlockObj.targetBegin
        successIndex = "NA"
        for i in range(0,len(synBlockObj.quearySeq)):
            if synBlockObj.quearySeq[i] != "-":
                qCoord += 1
                if which == "queary":
                    if qCoord == point:
                        successIndex = i
            ###################################
            if synBlockObj.targetSeq[i] != "-":
                tCoord += 1
                if which == "target":
                    if tCoord == point:
                        successIndex = i
            ########################
            if successIndex != "NA":
                break
        ########################
        #if successIndex == "NA":
        #    return "NA","NA","NA"
        ##########################################################################################
        ### Returns [ quearySeq,link,targetSeq ],[quearyName,quearyCoord],[targetName,targetCoord]
        ###############################################
        return [ synBlockObj.quearySeq[successIndex], \
                 synBlockObj.linkSeq[successIndex],\
                 synBlockObj.targetSeq[successIndex],
                 synBlockObj.orientation ], \
               [synBlockObj.quearyName,qCoord],\
               [synBlockObj.targetName,tCoord]
    ####################################
    ####################################
    ####################################
    elif synBlockObj.orientation == "-":
        if which == "queary":
            difference = synBlockObj.quearyEnd - point + 1
        elif which == "target":
            difference = point - synBlockObj.targetBegin
        #############################################################
        qCoord,tCoord = synBlockObj.quearyEnd,synBlockObj.targetBegin
        successIndex = "NA"
        for i in range(0,len(synBlockObj.quearySeq)):
            if synBlockObj.quearySeq[i] != "-":
                qCoord -= 1
                if which == "queary":
                    difference -= 1
            ###################################  
            if synBlockObj.targetSeq[i] != "-":
                tCoord += 1
                if which == "target":
                    difference -=1
            ###################
            if difference == 0:
                successIndex = i
                break
        ########################
        #if successIndex == "NA":
        #    return "NA","NA","NA"   
        ###############################################
        return [ synBlockObj.quearySeq[successIndex], \
                 synBlockObj.linkSeq[successIndex],\
                 synBlockObj.targetSeq[successIndex],
                 synBlockObj.orientation ], \
               [synBlockObj.quearyName,qCoord+1],\
               [synBlockObj.targetName,tCoord]

def quearyTree(chrom,point,treeDict,which="queary",method="first"):
    ### chrom,point = chromosome/position to look up in the interval tree
    ### treeDict = { chromosomeName:intervalTree() }
    ### which = "queary" or "target". This refers to the identity of the genome that the point belongs to. 
    ###         "queary" means the genome was used as the queary in the satsuma run. 
    ###         "target" means the genome was used as the target in the satsuma run. 
    
    ### This will return a few different things...
    ### syntenic coordinate found (or lack there of)
    
    ### The way Satsuma reports the information is with respect to the target genome that was used as input
    ### Therefore, all coordinates and sequences for the target genome are with respect
    ### to the (+) strand. The target genome acts like the reference genome for alignment
    ### So for any given alignment that is reported, the target sequence is ALWAYS 5'-3' positive strand with repsect to the target reference genome
    ### For an alignment where the orientation is (+), the queary sequence is also 5'-3' positive strand with repsect to the queary reference genome
    ### For an alignment where the orientation is (-), the queary sequence is RT with respect to the queary reference genome.
    ### The coordinates for both queary and target are again always with respect to the positive strand of the repsective reference genome.
    ### It is only the queary sequence that will need to be reverseTranscribed with repsect to its reference genome (but the coords remain the same).
    
    
    ### The information that is returned is in the form of 
    ### [ quearySequence,linkSequence,targetSequence,blockOrientation ] , [quearyName,quearyPoint] , [targetName,targetPoint]
    ### We return both the queary and target information so that a single function can be used to investigate the sytenny
    ### rather than two different ones for each queary/target genome
    ### Will return "NA","NA","NA" if no sytenic coordinate is found in the other genome
    
    ### Generate a list of syntenic blocks that this point overlaps ###
   
    synBlockIntervals = list(treeDict[chrom][point-1])
    blockCount = len(synBlockIntervals)
    ###################
    if blockCount == 1:
        a,q,t = quearySynBlock(synBlockIntervals[0].data,chrom,point,which=which)
    #####################
    elif blockCount == 0:
        a,q,t = "NA","NA","NA"
    #######################
    elif method == "first":
        a,q,t = quearySynBlock(synBlockIntervals[0].data,chrom,point,which=which)
    else:
        ### If there are more than one syntenicBlocks overlapping the position then we return "NA"
        ### This is the point in the algorithm in which a user defined function to determine the consensus sequence
        ### could be utilized or some other method to deal with these edge cases. I have written a function to come
        ### up with a consensus sequence, but how I define the consensus might be different from the users, so I
        ### have left this part open to interpretation. ~1.5% of randomly quearied positions appear to have multiple
        ### blocks overlapping them.
        a,q,t = "NA","NA","NA"
    ############
    return a,q,t

### Usage description/example

The program is written for 1 based indices. Any position in IGV that you queary should come back exactly the same if you are using the quearyTree function.  
Under the hood the quearyTree function will manually subtract 1 from the position, and use this to look up the overlapping sytnenicBlock(s). Then the original position will then be passed into the quearySynBlock function. 

But if the quearyTree function is not used, then any IGV coordinate you queary manually will come back as the position+1. So manually subtracting 1 from the original point is necessary when looking up the overlapping syntenicBlock(s), then the original point is used to obtain the coordinate.

**Using the quearyTree function**  
a,q,t = quearyTree(chrom,point,treeDict,which="queary",method="first")

**Manually quearying the intervalTree**  
blockMan = list(Q[chrom][**point-1**])[0].data  
a,q,t = quearySynBlock(blockMan,chrom,**point**,which="queary")

In [15]:
blockMan = list(Q["Chromosome_1_marmoratus"][128638491-1])[0].data
a,q,t = quearySynBlock(blockMan,"Chromosome_1_marmoratus",128638491,which="queary")
print(a,q,t)
a,q,t = quearyTree("Chromosome_1_marmoratus",128638491,Q,which="queary",method="first")
print(a,q,t)

['T', ' ', 'G', '+'] ['Chromosome_1_marmoratus', 128638491] ['ScHqM4t_1_gularis', 26917]
['T', ' ', 'G', '+'] ['Chromosome_1_marmoratus', 128638491] ['ScHqM4t_1_gularis', 26917]


### Recommended Usage with Pickle

To avoid redoing the computational work needed to create the tree each time it's needed, it is recommended to save it as a pickled object

In [ ]:
pickle.dump(query_tree, open("query_tree.p", "wb"))

The tree can then be quickly loaded into memory and queried as needed

In [ ]:
tree = pickle.load(open("query_tree.p", "rb"))

### Testing the code 

In [2]:
### Dumb/elegant way of assigning values to keys in a hashtable if the key doesn't exist ###
### Essentially done in a single line as oppossed to my old method of querying the hashtable twice, resulting in more lines and slower runtime ###
#counts = {}
#for i in range(0,100):
#    counts[1] = counts.get(1,0)+1
#print(counts)

In [ ]:
### For which=="queary" ###
which = "queary"
howMany = 100000
fail,success = 0,0

for c in marmDict:
    if c not in Q:
        continue
    for i in range(1,howMany):
        p = random.randint(1,len(marmDict[c]))
        seq,qInfo,tInfo = quearyTree(c,p,Q,which=which,metho="first")
        

In [41]:
### For which=="queary" ###
which = "queary"
howMany = 1000000
fail,success = 0,0

countDict = {}

for c in marmDict:
    if c not in Q:
        continue
    for i in range(1,howMany):
        p = random.randint(1,len(marmDict[c]))
        synBlockIntervals = list(Q[c][p-1])
        countDict[len(synBlockIntervals)] = countDict.get(len(synBlockIntervals),0)+1
        #if len(synBlockIntervals) == 8:
        #    for interval in synBlockIntervals:
        #        sObj = interval.data
        #        printInfo(sObj)
        #        print()
        #        printAln(sObj,step=200)
        #        print("###############")
        #        print("###############")
        #        print("###############")
        #        print("###############")
        #        print("###############")

        
        if len(synBlockIntervals) == 8:
            for interval in synBlockIntervals:
                sObj = interval.data
                a,q,t = quearySynBlock(sObj,c,p,which=which)
                print(a,sObj.length,sObj.pID)
            print("###############")
            print("###############")
            print("###############")
            print("###############")
            print("###############")
                
        
print(countDict)

['G', '|', 'G', '-'] 54 100.0
['C', '|', 'C', '+'] 54 100.0
['C', '|', 'C', '+'] 258 75.1938
['G', '|', 'G', '-'] 213 66.1972
['C', '|', 'C', '+'] 25 100.0
['G', '|', 'G', '-'] 287 78.0488
['G', '|', 'G', '-'] 54 100.0
['G', '|', 'G', '-'] 157 53.5032
###############
###############
###############
###############
###############
['G', '|', 'G', '-'] 54 100.0
['C', '|', 'C', '+'] 54 100.0
['C', '|', 'C', '+'] 258 75.1938
['G', '|', 'G', '-'] 213 66.1972
['C', '|', 'C', '+'] 25 100.0
['G', '|', 'G', '-'] 287 78.0488
['G', '|', 'G', '-'] 54 100.0
['G', '|', 'G', '-'] 157 53.5032
###############
###############
###############
###############
###############
['C', '|', 'C', '-'] 225 79.5556
['C', ' ', 'G', '-'] 202 69.3069
['C', ' ', '-', '-'] 182 64.8352
['C', ' ', 'T', '-'] 188 63.8298
['C', '|', 'C', '-'] 175 64.0
['C', '|', 'C', '-'] 214 65.8879
['C', '|', 'C', '-'] 250 56.0
['C', '|', 'C', '-'] 188 65.9574
###############
###############
###############
###############
##############

In [7]:
### For which=="queary" ###
which = "queary"
howMany = 100000
fail,success = 0,0

for c in marmDict:
    if c not in Q:
        continue
    for i in range(1,howMany):
        p = random.randint(1,len(marmDict[c]))
        synBlockIntervals = list(Q[c][p-1])
        
        
        if len(synBlockIntervals) == 1:
            sObj = synBlockIntervals[0].data

            a,q,t = (quearySynBlock(sObj,c,p,which=which))
            if a == "NA":
                continue
                
            ###########################
            if sObj.orientation == "-":
                quearyTest = revCompSatsuma(a[0])
            else:
                quearyTest = a[0]
            targetTest = a[2]
            #################
            

            ### Test for the "queary" information of the syntenic pair ###
            if q[1] != p:
                print("### Queary coordinate ERROR ###")
                print(c,p,q[1])
                printInfo(sObj)
                print()
                printAln(sObj)
            
            if quearyTest != marmDict[c][p-1]:
                print("### Queary coordinate/Sequence ERROR ###")
                print(c,p,q[1])
                printInfo(sObj)
                print()
                printAln(sObj)
                fail += 1
            
            ### Test for the "target" information of the syntenic pair ###
            if targetTest == "-":
                continue
            if targetTest != gulDict[t[0]][t[1]-1]:
                
                print("### Target coordinate/Sequence ERROR ###")
                print(c,p,t[1])
                print(t)
                print(a)
                print(gulDict[t[0]][t[1]-1])
                printInfo(sObj)
                print()
                printAln(sObj)
                fail += 1
            
            
            
            else:
                success += 1


print("Success "+str(success))
print("Fail "+str(fail))


Success 1548317
Fail 0


In [8]:
### For which=="target" ###
which = "target"
howMany = 10000
fail,success = 0,0

for c in gulDict:
    if c not in T:
        continue
    for i in range(1,howMany):
        p = random.randint(1,len(gulDict[c]))
        synBlockIntervals = list(T[c][p-1])
        
        
        if len(synBlockIntervals) == 1:
            sObj = synBlockIntervals[0].data

            a,q,t = (quearySynBlock(sObj,c,p,which=which))
            if a == "NA":
                continue
                
            ###########################
            if sObj.orientation == "-":
                quearyTest = revCompSatsuma(a[0])
            else:
                quearyTest = a[0]
            targetTest = a[2]
            #################
            
            ### Test for the "queary" information of the syntenic pair ###
            if t[1] != p:
                print("### Target coordinate ERROR ###")
                print(c,p,t[1])
                printInfo(sObj)
                print(a)
                print()
                printAln(sObj)
            
            if targetTest != gulDict[c][p-1]:
                print("### Target coordinate/Sequence ERROR ###")
                print(c,p,t[1])
                printInfo(sObj)
                print(a)
                print()
                printAln(sObj)
                fail += 1
            
            ### Test for the "target" information of the syntenic pair ###
            if quearyTest == "-":
                continue
            if quearyTest != marmDict[q[0]][q[1]-1]:
                
                print("### Queary coordinate/Sequence ERROR ###")
                print(c,p,q[1])
                print(t)
                print(a)
                print(gulDict[q[0]][q[1]-1])
                printInfo(sObj)
                print()
                prin1tAln(sObj)
                fail += 1
            
            
            
            else:
                success += 1


print("Success "+str(success))
print("Fail "+str(fail))

Success 987976
Fail 0
